In [49]:
%load_ext autoreload
%autoreload 2
import os

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
# make a folder to save output images
from unet import UNet
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

# a function used to transform numpy array to image format
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

#################### select your hyperparameters ############################
num_epochs = 10
batch_size = 4
n_samples = 20

####### define image transforms, you can have other choices, explore it! #####
transform = transforms.ToTensor()

# download MNIST dataset
dataset = MNIST('./data', transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [51]:
# set the model to be the autoencoder defined above
model = UNet(n_channels=1, n_classes=1)
############ chose appropriate loss function ######################
criterion = nn.MSELoss()

# set the optimizer, explore the effect of different optimizers
lr = 0.1
optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=1e-8)

In [52]:
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        if i > n_samples:
            break
        img, _ = data
        #img = img.view(img.size(0), -1)
        # ===================forward=====================
        output = model(img)
        target = img
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================user interaction========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data))
    if epoch % 1 == 0:
        pic = to_img(output.data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))
        
        ori_pic = to_img(img.data)
        save_image(ori_pic, './mlp_img/ori_image_{}.png'.format(epoch))

epoch [1/10], loss:0.3185
epoch [2/10], loss:0.1247
epoch [3/10], loss:0.0711
epoch [4/10], loss:0.1151
epoch [5/10], loss:0.0449
epoch [6/10], loss:0.0674
epoch [7/10], loss:0.0654
epoch [8/10], loss:0.0561
epoch [9/10], loss:0.0805
epoch [10/10], loss:0.0307
